**Import required libraries**

In [ ]:
import intake
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import matplotlib.pyplot as plt
import datetime
import numpy as np

**Open the NWM intake catalog**

In [ ]:
catalog = intake.open_catalog('s3://nextgen-dmac-cloud-ingest/nwm_ll/nwm_intake.yml',storage_options={'anon':True})
(list(catalog))

### Compare NWM netCDF source files with Kerchunked virtual 'best timeseries' aggregations

Part of the processing included adding additional coordinate and attribute values to the souce netCDF files produced by NWM.  Below is a list of the additions.  The following cells open example .nc files for each forecast type, and then the equivalent Kerchunk aggregation from the Intake catalog.


**Coordinates** added to the kerchunked datasets include: 
* latitude
* longitude

**Variables** added include:
* altitude: altitude of node 
* link_id: the link id of the node
* from_link_id: the from_link_id of the node in the network topology
* to_link_id: the to_link_id of the node in the network topology
* terminal_node: whether this node is a terminal node or not


**Dimensions**
Other changes include to the length of the `time` dimension in the dataset, which is increased to the # of forecast steps in the best timeseries aggregations, and the data variables all vary on (time, featureid) rather than only featureid as in the source forecast files.


In [ ]:
## open short range dataset:
import xarray as xr

ds_sr_nc = xr.open_dataset("files/nwm.t23z.short_range.channel_rt.f001.conus.nc")
ds_sr_nc

In [ ]:
cat = catalog['NWM_Best_CONUS_Short_Range']
ds_sr = cat.to_dask()
ds_sr

In [ ]:
# medium range conus:
ds_mr_nc = xr.open_dataset("files/nwm.t18z.medium_range_blend.channel_rt.f001.conus.nc")
ds_mr_nc

In [ ]:
cat = catalog['NWM_Best_CONUS_Medium_Range']
ds_mr = cat.to_dask()
ds_mr

In [ ]:
# short range alaska:
ds_ak_sr_nc = xr.open_dataset("files/nwm.t00z.short_range.channel_rt.f001.alaska.nc")
ds_ak_sr_nc

In [ ]:
cat = catalog['NWM_Best_Alaska_Short_Range']
ds_ak_sr = cat.to_dask()
ds_ak_sr

In [ ]:
# medium range alaska:
ds_ak_mr_nc = xr.open_dataset("files/nwm.t00z.medium_range_blend.channel_rt.f001.alaska.nc")
ds_ak_mr_nc

In [ ]:
cat = catalog['NWM_Best_Alaska_Medium_Range']
ds_ak_mr = cat.to_dask()
ds_ak_mr

In [ ]:
# short range hawaii:
ds_hi_sr_nc = xr.open_dataset("files/nwm.t00z.short_range.channel_rt.f00015.hawaii.nc")
ds_hi_sr_nc

In [ ]:
cat = catalog['NWM_Best_Hawaii_Short_Range']
ds_hi_sr = cat.to_dask()
ds_hi_sr

In [ ]:
# short range puerto rico:
ds_pr_sr_nc = xr.open_dataset("files/nwm.t06z.short_range.channel_rt.f001.puertorico.nc")
ds_pr_sr_nc

In [ ]:
cat = catalog['NWM_Best_PuertoRico_Short_Range']
ds_pr_sr = cat.to_dask()
ds_pr_sr

#### Output the feature_id array subset for the netCDF sources:

In [ ]:

feature_id_sr_nc = np.random.choice(ds_sr_nc.isel(time=0).coords['feature_id'], size=100)
feature_id_mr_nc = np.random.choice(ds_mr_nc.isel(time=0).coords['feature_id'], size=100)
feature_id_ak_sr_nc = np.random.choice(ds_ak_sr_nc.isel(time=0).coords['feature_id'], size=100)
feature_id_ak_mr_nc = np.random.choice(ds_ak_mr_nc.isel(time=0).coords['feature_id'], size=100)
feature_id_hi_sr_nc = np.random.choice(ds_hi_sr_nc.isel(time=0).coords['feature_id'], size=100)
feature_id_pr_sr_nc = np.random.choice(ds_pr_sr_nc.isel(time=0).coords['feature_id'], size=100)

print(f"feature_id_sr_nc: {feature_id_sr_nc}\n\nfeature_id_mr_nc: {feature_id_mr_nc}\n\nfeature_id_ak_sr_nc: {feature_id_ak_sr_nc} \
\n\nfeature_id_ak_mr_nc: {feature_id_ak_mr_nc}\n\nfeature_id_hi_sr_nc: {feature_id_hi_sr_nc}\n\nfeature_id_pr_sr_nc: {feature_id_pr_sr_nc}")

#### Output the feature_id array subset for the kerchunk aggregation sources:

This should yield similar results to the above (all feature_id values present in the collection arrays)

In [ ]:
feature_id_sr = np.random.choice(ds_sr.isel(time=0).coords['feature_id'], size=100)
feature_id_mr = np.random.choice(ds_mr.isel(time=0).coords['feature_id'], size=100)
feature_id_ak_sr = np.random.choice(ds_ak_sr.isel(time=0).coords['feature_id'], size=100)
feature_id_ak_mr = np.random.choice(ds_ak_mr.isel(time=0).coords['feature_id'], size=100)
feature_id_hi_sr = np.random.choice(ds_hi_sr.isel(time=0).coords['feature_id'], size=100)
feature_id_pr_sr = np.random.choice(ds_pr_sr.isel(time=0).coords['feature_id'], size=100)

print(f"feature_id_sr: {feature_id_sr}\n\nfeature_id_mr: {feature_id_mr}\n\nfeature_id_ak_sr: {feature_id_ak_sr} \
\n\nfeature_id_ak_mr: {feature_id_ak_mr}\n\nfeature_id_hi_sr: {feature_id_hi_sr}\n\nfeature_id_pr_sr: {feature_id_pr_sr}")

### Plots of different forecast collections

First, assign a time range of a two week window from today's date for plotting:

In [ ]:
# create a time window of +-1 week
start_t = datetime.datetime.now() - datetime.timedelta(days=7)
end_t = datetime.datetime.now() + datetime.timedelta(days=7)

# create a time window of +-1 day
#start_t = datetime.datetime.now() - datetime.timedelta(days=1)
#end_t = datetime.datetime.now() + datetime.timedelta(days=1)


Check a few random feature_id index values from CONUS Short and Medium range collections to verify valid feature_ids returned:

In [ ]:
#feature_id = ds_sr.isel(time=0).coords['feature_id'].values[0]
feature_id = np.random.choice(ds_sr.isel(time=0).coords['feature_id'])

#print output:
print(f"start_t: {start_t}, end_t: {end_t}, feature_id: {feature_id}")


In [ ]:
#feature_id = ds_mr.isel(time=0).coords['feature_id'].values[0]
feature_id = np.random.choice(ds_mr.isel(time=0).coords['feature_id'])

#print output:
print(f"start_t: {start_t}, end_t: {end_t}, feature_id: {feature_id}")


**NWM_Best_CONUS_Short_Range**, random feature_id:

In [ ]:
%%time

feature_id = np.random.choice(ds_sr.isel(time=0).coords['feature_id'])
ts_ds = ds_sr.sel(time=slice(start_t, end_t),feature_id=feature_id)
plt.figure(figsize=(10,6))
plt.plot(ts_ds.time, ts_ds.streamflow[:])

print(f"start_t: {start_t}, end_t: {end_t}, feature_id: {feature_id}")
plt.show()

**NWM_Best_CONUS_Medium_Range**, random feature_id:

In [ ]:
%%time

feature_id = np.random.choice(ds_mr.isel(time=0).coords['feature_id'])
ts_ds = ds_mr.sel(time=slice(start_t, end_t),feature_id=feature_id)
plt.figure(figsize=(10,6))
plt.plot(ts_ds.time, ts_ds.streamflow[:])

print(f"start_t: {start_t}, end_t: {end_t}, feature_id: {feature_id}")
plt.show()

**NWM_Best_PuertoRico_Short_Range**, random feature_id:

In [ ]:
%%time

feature_id = np.random.choice(ds_pr_sr.isel(time=0).coords['feature_id'])
ts_ds = ds_pr_sr.sel(time=slice(start_t, end_t),feature_id=feature_id)
plt.figure(figsize=(10,6))
plt.plot(ts_ds.time, ts_ds.streamflow[:])

print(f"start_t: {start_t}, end_t: {end_t}, feature_id: {feature_id}")
plt.show()

In [ ]:
%%time

feature_id = np.random.choice(ds_pr_sr.isel(time=0).coords['feature_id'])
ts_ds = ds_pr_sr.sel(time=slice(start_t, end_t),feature_id=feature_id)
plt.figure(figsize=(10,6))
plt.plot(ts_ds.time, ts_ds.streamflow[:])

print(f"start_t: {start_t}, end_t: {end_t}, feature_id: {feature_id}")
plt.show()

**NWM_Best_Hawaii_Short_Range**, random feature_id:

In [ ]:
%%time

feature_id = np.random.choice(ds_hi_sr.isel(time=0).coords['feature_id'])
ts_ds = ds_hi_sr.sel(time=slice(start_t, end_t),feature_id=feature_id)
plt.figure(figsize=(10,6))
plt.plot(ts_ds.time, ts_ds.streamflow[:])

print(f"start_t: {start_t}, end_t: {end_t}, feature_id: {feature_id}")
plt.show()

**NWM_Best_Alaska_Short_Range**, random feature_id:

In [ ]:
%%time

feature_id = np.random.choice(ds_ak_sr.isel(time=0).coords['feature_id'])
ts_ds = ds_ak_sr.sel(time=slice(start_t, end_t),feature_id=feature_id)
plt.figure(figsize=(10,6))
plt.plot(ts_ds.time, ts_ds.streamflow[:])

print(f"start_t: {start_t}, end_t: {end_t}, feature_id: {feature_id}")
plt.show()

**NWM_Best_Alaska_Medium_Range**, random feature_id:

In [ ]:
%%time

feature_id = np.random.choice(ds_ak_mr.isel(time=0).coords['feature_id'])
ts_ds = ds_ak_mr.sel(time=slice(start_t, end_t),feature_id=feature_id)
plt.figure(figsize=(10,6))
plt.plot(ts_ds.time, ts_ds.streamflow[:])

print(f"start_t: {start_t}, end_t: {end_t}, feature_id: {feature_id}")
plt.show()

### Original Plot

This is the equivalent of the plot code in the original notebook, with a different two week time window to match the above plots

&nbsp;

**Extract and plot the timeseries data**

In [ ]:
# use the pre-selected feature_id instead:

# this will select the first feature_id value at time slice 0:
#feature_id = ds_ak_mr.isel(time=0).coords['feature_id'].values[0]

# this will select the feature_id value at index -100 at time slice 0:
feature_id = ds_ak_mr.isel(time=0).coords['feature_id'].values[-100]

# this will use the hardcoded feature_id from the original notebook (sometimes errors out if not present):
feature_id = ds_ak_mr.sel(feature_id=19020190088835).coords['feature_id'].values

#print output:
print(f"start_t: {start_t}, end_t: {end_t}, feature_id: {feature_id}")


ts_ds = ds_ak_mr.sel(time=slice(start_t, end_t),feature_id=feature_id)
plt.figure(figsize=(10,6))
plt.plot(ts_ds.time, ts_ds.streamflow[:])

plt.show()

### Testing 

Just some testing of xarray subset functions for the above slicing/subsetting/plotting tests:

In [ ]:
# testing xarray functions............

#feature_id = 19020190088835
#start_t='2024-02-23T13:00'
#end_t = '2024-03-16T02:00'



# slice by first feature_id element:

# these functions slice by the first feature_id value, returning an xarray dataset with all times at the single feature_id index value
#feature_id = ds_mr.isel(feature_id=0)
#feature_id = ds_mr.isel(feature_id=0).coords['feature_id']

# to get the actual value of the feature_id coordinate here, we don't use array syntax, as it's a scalar:
feature_id = ds_mr.isel(feature_id=0).coords['feature_id'].values


# slice by first element of the time dimension:

# returns full xarray dataset:
#feature_id = ds_mr.isel(time=0)

# returns the feature_id coordinate xarray datarray:
#feature_id = ds_mr.isel(time=0).coords['feature_id']
feature_id = ds_mr.isel(time=0).coords['feature_id'].values

# returns the 0th feature_id coordinate value:
#feature_id = ds_mr.isel(time=0).coords['feature_id'].values[0]



# or, just select a feature_id (doesn't actually do anything just output the same value):

#feature_id = ds_sr.sel(feature_id=19020190088835)
#feature_id = ds_ak_mr.sel(feature_id=19020190088835).coords['feature_id'].values




# np random sample of 1000 feature_ids:
#feature_id_sr = np.random.choice(ds_sr.isel(time=0).coords['feature_id'], size=100)
feature_id = np.random.choice(ds_sr.isel(time=0).coords['feature_id'], size=100)
#feature_id = np.random.choice(ds_mr.isel(time=0).coords['feature_id'], size=100)
#feature_id = np.random.choice(ds_ak_sr.isel(time=0).coords['feature_id'], size=100)
#feature_id = np.random.choice(ds_ak_mr.isel(time=0).coords['feature_id'], size=100)
#feature_id = np.random.choice(ds_hi_sr.isel(time=0).coords['feature_id'], size=100)
#feature_id = np.random.choice(ds_pr_sr.isel(time=0).coords['feature_id'], size=100)

#print(feature_id_sr)
#print(feature_id)